In [1]:
%load_ext lab_black

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np
import re
import math
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from textblob import Word
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
    HashingVectorizer,
)
from lightgbm import LGBMRegressor

/Users/kdang/anaconda3/envs/kaggle/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
train = pd.read_excel("Data_Train.xlsx")
test = pd.read_excel("Data_Test.xlsx")
train["istrain"] = 1
test["istrain"] = 0
traintest = pd.concat([train, test], axis=0)

In [3]:
train.shape, test.shape, traintest.shape

((7628, 3), (2748, 2), (10376, 3))

In [4]:
train.T.head()

,0,1,2,3,4,5,6,7,8,9,...,7618,7619,7620,7621,7622,7623,7624,7625,7626,7627
STORY,But the most painful was the huge reversal in ...,How formidable is the opposition alliance amon...,Most Asian currencies were trading lower today...,"If you want to answer any question, click on ‘...","In global markets, gold prices edged up today ...",BEIJING: Chinese tech giant Huawei has announc...,Mumbai: India Inc's external commercial borrow...,"On Wednesday, Federal Reserve Chairman Jerome ...",What more can you give to the audience? I have...,"com, Arbaaz Khan spoke about getting back to D...",...,"With up to 14.7 million pixels, one billion co...",A BJD supporter in Puri. Congress leaders as w...,Bollywood celebrities took to social media to ...,"However, as of now, there is no confirmation a...","In terms of optics, the back of the Redmi Note...","Karnataka has been a Congress bastion, but it ...","The film, which also features Janhvi Kapoor, w...",The database has been created after bringing t...,"The state, which has had an uneasy relationshi...","Virus stars Kunchacko Boban, Tovino Thomas, In..."
SECTION,3,0,3,1,3,1,3,3,2,2,...,1,0,2,1,1,0,2,1,0,2
istrain,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [5]:
test.T.head()

,0,1,2,3,4,5,6,7,8,9,...,2738,2739,2740,2741,2742,2743,2744,2745,2746,2747
STORY,2019 will see gadgets like gaming smartphones ...,It has also unleashed a wave of changes in the...,It can be confusing to pick the right smartpho...,The mobile application is integrated with a da...,We have rounded up some of the gadgets that sh...,"""Imagine if every message you sent was kept wi...",Positioned along the four sides of the Asus RO...,"In fact, when I applied to USC film school the...","As spotted by Android Police, Netflix is testi...",Her moves were immaculately choreographed as s...,...,The traffic challan payment is also restricted...,"“We are not like Western countries, where peop...","Further, SBICAP Securities says delivery of th...","Two, Raj Thackeray’s Maharashtra Navnirman Sen...",Senior leaders of the BJP are using the sugges...,"According to researchers, fraud in the mobile ...",The iPhone XS and XS Max share the Apple A12 c...,"On the photography front, the Note 5 Pro featu...",UDAY mandated that discoms bring the gap betwe...,Ripple also helps bank customers send money to...
istrain,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pipeline = Pipeline([
    ('text', FunctionTransformer(lambda x: x.lower())),
    ('')
])